# Setup

In [3]:
!git clone https://github.com/jefflai108/6.438-final.git
!git pull

fatal: destination path '6.438-final' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git


In [6]:
import os
os.chdir('6.438-final')
!pip install -r requirements.txt

In [0]:
from transformers.modeling_gpt2 import GPT2LMHeadModel
# This downloads GPT-2 Medium, it takes a little while
_ = GPT2LMHeadModel.from_pretrained("gpt2-medium")
from run_pplm import run_pplm_example

# Mount Pretrained Models on Drive
https://stackoverflow.com/questions/46986398/import-data-into-google-colaboratory
https://colab.research.google.com/drive/1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q#scrollTo=c99EvWo1s9-x

In [66]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# PPLM Examples


In [0]:
run_pplm_example(
    cond_text="The potato",
    num_samples=3,
    bag_of_words='military',
    length=50,
    stepsize=0.03,
    sample=True,
    num_iterations=3,
    window_length=5,
    gamma=1.5,
    gm_scale=0.95,
    kl_scale=0.01,
    verbosity='regular'
)

In [0]:
run_pplm_example(
    cond_text="Once upon a time",
    num_samples=10,
    discrim='sentiment',
    class_label='very_positive',
    length=50,
    stepsize=0.05,
    sample=True,
    num_iterations=10,
    gamma=1,
    gm_scale=0.9,
    kl_scale=0.02,
    verbosity='quiet'
)

= Prefix of sentence =
<|endoftext|>My dog died

= Unperturbed generated text =
<|endoftext|>My dog died in February, after suffering from severe arthritis. He had been suffering with a terrible cold that was causing his skin to break. I couldn't afford a replacement dog and couldn't afford to have him taken to the vet. I knew the vet would be

= Perturbed generated text 1 =
<|endoftext|>My dog died.

He was a beautiful, loving, loyal, loving, beautiful soul. He did what I loved and wanted. I'm going to miss him. I'm going to miss you. I know you will always miss him. I know he



# Set up Evaluation Methods

Evaluation with Perplexity
Perplexity with a GPT-2:
https://www.reddit.com/r/LanguageTechnology/comments/bucn53/perplexity_score_of_gpt2/
https://github.com/huggingface/transformers/issues/4147

In [0]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
import torch
import math

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
config = GPT2Config.from_pretrained('gpt2')
per_model = GPT2LMHeadModel.from_pretrained('gpt2', config=config)
per_model.eval() # no gradients
if torch.cuda.is_available():
    device = 'cuda'
per_model.to(device) # run on GPU

def perplexity_score(sentence, model, tokenizer):
    # Encode a text inputs
    indexed_tokens = tokenizer.encode(sentence)
    # Convert indexed tokens in a PyTorch tensor
    input_ids = torch.tensor([indexed_tokens]) #.unsqueeze(0) 
    # If you have a GPU, put everything on cuda
    input_ids = input_ids.to('cuda')

    # Predict all tokens
    with torch.no_grad():
        outputs = model(input_ids, labels = input_ids)
    loss, logits = outputs[:2]

    return math.exp(loss)

Evaluation with Distinct-1,2,3
https://github.com/XinnuoXu/mmi_anti_pytorch/blob/master/diversity.py

In [0]:
SAMPEL_TIMES = 2 # sampled from the entire corpus
def diversity(sentences):
        import sys
        import random
        line_list = []
        for line in sentences:
                line_list.append(line.strip())
        d1, d2, d3 = 0.0, 0.0, 0.0
        for _ in range(0, SAMPEL_TIMES):
                uni_set, bi_set, tri_set = set(), set(), set()
                uni_num, bi_num, tri_num = 0, 0, 0
                for line in random.sample(line_list, min(2000, len(line_list))):
                        #print(line)
                        flist = line.split(" ")
                        for x in flist:
                                uni_set.add(x)
                                uni_num += 1
                        for i in range(0, len(flist)-1):
                                bi_set.add(flist[i] + "<XXN>" + flist[i + 1])
                                bi_num += 1
                        for j in range(0, len(flist)-2):
                                tri_set.add(flist[j] + "<XXN>" + flist[j + 1] + 
                                            "<XXN>" + flist[j + 2])
                                tri_num += 1
                d1 += len(uni_set) / float(uni_num)
                d2 += len(bi_set)  / float(bi_num)
                d3 += len(tri_set) / float(tri_num)
        print("DIVERSE-1", d1 / SAMPEL_TIMES)
        print("DIVERSE-2", d2 / SAMPEL_TIMES)
        print("DIVERSE-3", d3 / SAMPEL_TIMES)
        print("DISTINCT SENTENCES", len(set(line_list)) / float(len(line_list)))

test evaluation methods

In [2]:
def evaluate_all(sentences):
    print('Evaluate by perplexity')
    print([perplexity_score(text, per_model, tokenizer) for text in sentences])
    print('\n')
    print('Evaluate by diversity')
    diversity(sentences)

a=["""My dog died in February, after suffering from severe arthritis. He had been suffering with a terrible cold that was causing his skin to break. I couldn't afford a replacement dog and couldn't afford to have him taken to the vet. I knew the vet would be""",
  """My dog died after getting stuck in a tree... I don't wish this to happen to my favorite character from one of my favorite movies, \"The Magnificent Seven." "It's all sad when someone dies so well." The Magnificent Seven movie was""",
  """My name is Jeff. How are you?"""]
evaluate_all(a)

Evaluate by perplexity


NameError: ignored

# Generate examples with our methods 


In [0]:
os.chdir('ours')

loop over prefix and conditions and log the generated sentences and evaluation scores

In [0]:
for condition in ["negative", "positive"]:
    for prefix in ["The following is a negative sentence. The chicken tastes", "The chicken tastes"]:
        print(condition, prefix)
        a = !python generate.py --prefix prefix --condition condition --num_samples 2 --length 50

negative The following is a negative sentence. The chicken tastes
^C
negative The chicken tastes


In [63]:
for i in a:
    print(i)

. The chicken tastes bad. But then he goes "oh, the meat was great" – I am just saying that the meat was great, but the flavor was so…off, it was almost as much as it tasted
. The chicken tastes so bad, he has tried every dish with only one success: the meat. The meat in these dishes was maroon, with some brown. This is a sign that the meat has already eaten all it


test run --> loop this over ALL conditioned text --> probably store them in dictionaries; so during evaluation the printouts are prettier

In [0]:
from run_pplm_evaluation import run_pplm_evaluation
generated_texts = run_pplm_evaluation(
    cond_text="Once upon a time",
    num_samples=5,
    discrim='sentiment',
    class_label='very_positive',
    length=50,
    stepsize=0.05,
    sample=True,
    num_iterations=10,
    gamma=1,
    gm_scale=0.9,
    kl_scale=0.02,
    verbosity='quiet'
)

Compute the perplexity for original unperturbed and PPLM perturbed texts

In [0]:
import numpy as np 

print('Evaluate Perplexity with GPT-2')
print(generated_texts)
# get rid of the beginning '<|endoftext|>'
processed_texts = [sample.replace('<|endoftext|>','') for sample in generated_texts]
print(processed_texts)
print('')
print('Original perplexity score: %f' % perplexity_score(processed_texts[0], model, tokenizer))
print('')
avg_pplm_ppl = np.mean([perplexity_score(text, model, tokenizer) for text in processed_texts[1:]])
print('PPLM perplexity score over %d samples: %f' % (len(processed_texts[1:]), avg_pplm_ppl))

Evaluate Perplexity with GPT-2


NameError: ignored

#Evaluation with Sentiment Classification 


In [0]:
import torch

class ClassificationHead(torch.nn.Module):
    """Classification Head for  transformer encoders"""

    def __init__(self, class_size, embed_size):
        super(ClassificationHead, self).__init__()
        self.class_size = class_size
        self.embed_size = embed_size
        # self.mlp1 = torch.nn.Linear(embed_size, embed_size)
        # self.mlp2 = (torch.nn.Linear(embed_size, class_size))
        self.mlp = torch.nn.Linear(embed_size, class_size)

    def forward(self, hidden_state):
        # hidden_state = F.relu(self.mlp1(hidden_state))
        # hidden_state = self.mlp2(hidden_state)
        logits = self.mlp(hidden_state)
        return logits

model = ClassificationHead(class_size=5, embed_size=1024).to(device)
model.load_state_dict(torch.load('paper_code/discrim_models/sentiment_classifierhead.pt', map_location=device))
model.eval()

ClassificationHead(
  (mlp): Linear(in_features=1024, out_features=5, bias=True)
)

In [0]:
for line in processed_texts:
    model(line)

AttributeError: ignored

# Train up an Sentiment Analysis Model on Twitter
https://www.kaggle.com/paoloripamonti/twitter-sentiment-analysis

In [0]:
!pip install gensim --upgrade
!pip install keras --upgrade
!pip install pandas --upgrade

In [0]:
# DataFrame
import pandas as pd

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

nltk.download('stopwords')

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Settings

In [0]:
# Setting 
# DATASET
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
TRAIN_SIZE = 0.8

# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

# KERAS
SEQUENCE_LENGTH = 300
EPOCHS = 8
BATCH_SIZE = 1024

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)

# EXPORT
KERAS_MODEL = "model.h5"
WORD2VEC_MODEL = "model.w2v"
TOKENIZER_MODEL = "tokenizer.pkl"
ENCODER_MODEL = "encoder.pkl"

Dataset details

* target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
* ids: The id of the tweet ( 2087)
* date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)
* flag: The query (lyx). If there is no query, then this value is NO_QUERY.
* user: the user that tweeted (robotickilldozr)
* text: the text of the tweet (Lyx is cool)

In [0]:
dataset_filename = os.listdir("../input")[0]
dataset_path = os.path.join("..","input",dataset_filename)
print("Open file:", dataset_path)
df = pd.read_csv(dataset_path, encoding =DATASET_ENCODING , names=DATASET_COLUMNS)

print("Dataset size:", len(df))

df.head(5)

FileNotFoundError: ignored

Word2Vec

In [0]:
w2v_model = gensim.models.word2vec.Word2Vec.load("ours/model.w2v")

2020-05-06 16:46:42,514 : INFO : loading Word2Vec object from ours/model.w2v


FileNotFoundError: ignored

Tokenizer

In [0]:
import os 
print(os.getcwd())

/content


# CTRL: 